<a href="https://colab.research.google.com/github/Gilf641/EPAi/blob/master/Session10_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
!pip install faker

In [ ]:
from collections import namedtuple
from collections import defaultdict
from faker import Faker
from datetime import date, datetime, timezone
from time import perf_counter, localtime
from functools import wraps
import random
import pandas as pd
from time import perf_counter

## Validation Tests

In [ ]:
# Validation Tests
def validate_input(n):
	if not isinstance(n, int):
		raise ValueError


def validate_input_type(name, birthdate, sex, blood_group, current_location):
	if not isinstance(name, str):
		raise ValueError
	if not isinstance(birthdate, datetime):
		raise ValueError
	if not isinstance(sex, str):
		raise ValueError
	if not isinstance(blood_group, str):
		raise ValueError
	if not isinstance(current_location, tuple):
		raise ValueError


def validate_test_case(inp: list):
	if len(inp)<5:
		raise ValueError

def validate_reps(n):
	if not isinstance(n, int):
		raise ValueError

def validate_prop_input(n):
	if n < 100:
		raise ValueError

## Decorator

In [ ]:
fake = Faker('en_IN')
random_profile = namedtuple('Profile', ['name', 'age', 'sex', 'blood_group', 'location'])


# lambda func to calculate age
calculate_age = lambda birthdate: int((date.today() - birthdate).days / 365.2425)


# REP DECORATOR
def reps(n: int):
    def timed(fn):
        def inner(*args, **kwargs):
            total_elapsed = 0
            validate_reps(n)
            for i in range(n):
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)

            avg_run_time = total_elapsed / n
            print(f'Average Run time of {fn.__name__} is : {avg_run_time} seconds for {n} reps.')
            return result
        return inner
    return timed

## Task 1

In [ ]:
profile_list = []

fake_profiles = namedtuple('ProfileDetails', 'profile')
test_ip = ['Rohan', datetime(1998,10,8), 'M', 'A+', (140,30)] 

@reps(2)
def generate_random_profile(n: int, test=False, test_input=[]):
	'''
	Generates Random Profile using NamedTuple
	Args:
		n:- Number of Profiles to Generate
	'''
	validate_input(n)
	blood_type = defaultdict(lambda: 0)

	if test:
		validate_test_case(test_input)
		name, birthdate, sex, blood_group, current_location = test_input
		validate_input_type(name, birthdate, sex, blood_group, current_location)
		profile = random_profile(name, calculate_age(birthdate), sex, blood_group, current_location)
		profile_list.append(profile)

	else:
		for i in range(n):
			a = fake.profile()
			profile = random_profile(a['name'], calculate_age(a['birthdate']), a['sex'], a['blood_group'], a['current_location'])
			profile_list.append(profile)
			blood_type[profile.blood_group] +=1 


	return f'Summary of {n} Random Profiles: \
			\nThe avg age is {sum([i.age for i in profile_list])/len(profile_list)} years old.\
			 \nThe oldest person alive is of {max([i.age for i in profile_list])} years. \
			 \nThe average latitude & longitude value is ({sum([i.location[0] for i in profile_list])/len(profile_list)}, ({sum([i.location[1] for i in profile_list])/len(profile_list)}) Degrees.\
			 \nThe Largest Blood Type is {sorted(blood_type.items(), key=lambda x: x[1], reverse=True)[0]}'


print(generate_random_profile(10000))

Average Run time of generate_random_profile is : 20.28163247499998 for 2 reps.
Summary of 10000 Random Profiles: 			
The avg age is 57.3953 years old.			 
The oldest person alive is of 115 years. 			 
The average latitude & longitude value is (0.6805312445, (-0.3984424349) Degrees.			 
The Largest Blood Type is ('O-', 1301)


## Task 2

In [ ]:
profile_list2 = []

blood_type = defaultdict(lambda: 0)

@reps(2)
def generate_random_profile_dicts(n: int):
	'''
	Generates Random Profile using Dictionary
	Args:
		n:- Number of Profiles to Generate
	'''

	validate_input(n)
	print('-'*50)


	for i in range(n):
		profile_dict = defaultdict(lambda :0)
		a = fake.profile()
		keys = ['name', 'birthdate', 'sex', 'blood_group', 'current_location']
		for key in keys:
			profile_dict[key] = a[key]

		keys = ['name', 'sex', 'blood_group']
		for key in keys:
			profile_dict[key] = a[key]

		location = a['current_location']
		age = calculate_age(a['birthdate'])
		
		profile_dict['location']  = location
		profile_dict['age'] = age

		blood_type[profile_dict['blood_group']] += 1

		profile_list2.append(profile_dict)

	avg_age = sum([i['age'] for i in profile_list2])/len(profile_list2)
	max_age = max([i['age'] for i in profile_list2])
	avg_lat, avg_long = sum([i['location'][0] for i in profile_list2])/len(profile_list2), sum([i['location'][1] for i in profile_list2])/len(profile_list2)

	return f'Summary of {n} Random Profiles: \
			\nThe avg age is {avg_age} years old.\
			 \nThe oldest person alive is of {max_age} years. \
			 \nThe average latitude & longitude value is ({avg_lat}, {avg_long}) Degrees.\
			 \nThe Largest Blood Type is {sorted(blood_type.items(), key=lambda x: x[1], reverse=True)[0]}'


	
print(generate_random_profile_dicts(10000))

--------------------------------------------------
--------------------------------------------------
Average Run time of generate_random_profile_dicts is : 20.102740623499983 for 2 reps.
Summary of 10000 Random Profiles: 			
The avg age is 57.76195 years old.			 
The oldest person alive is of 115 years. 			 
The average latitude & longitude value is (-0.0804716565, -0.29704250105) Degrees.			 
The Largest Blood Type is ('A-', 2564)


# namedtuple is Faster than Dictionary!

## Task 3

In [ ]:
symbol_generator =  lambda x: str(x[:2] + x[-2:]).upper() if len(x.split()) == 1 else str(x.split()[0][:2]+x.split()[1][:2]).upper()

companies = namedtuple('StockMarket', ['name', 'symbol', 'open', 'low', 'high', 'close'])

weight_ = []
company_stock_profiles = []

def stock_exchange(n):
	'''
	Generates Company Stock Profiles
	Args:
		N = Number of profiles to be generated.
	'''
	validate_input(n)
	validate_prop_input(n)
	
	for i in range(n):
		name_ = fake.company()

		symbol_ = symbol_generator(name_)

		open_value = random.randint(100, 4000)
		comp_weight = round(random.uniform(0,0.7),3)

		weight_.append(comp_weight)

		comp_contrib = round(open_value * comp_weight/sum(weight_), 3)

		high_value = round(random.uniform(0.8, 1.0) * open_value * comp_weight/sum(weight_), 3)
		close_value = round(random.uniform(0.5,0.7) * open_value * comp_weight/sum(weight_), 3)
		low_value = round(random.uniform(0.35,0.4) * open_value * comp_weight/sum(weight_), 3)

		comp_profile = companies(name_, symbol_, open_value, low_value, high_value, close_value)
		company_stock_profiles.append(comp_profile)


	Total = pd.DataFrame({'Company Name': ['GRAND TOTAL'] ,
					'Symbol': [' '],
					'Open': ['= '+str(round(sum([i.open for i in company_stock_profiles]), 3))], 
					'Low': ['= '+str(round(sum([i.low for i in company_stock_profiles]), 3))],
					'High': ['= '+str(round(sum([i.high for i in company_stock_profiles]), 3))],
					'Close': ['= '+str(round(sum([i.close for i in company_stock_profiles]), 3))]
							})

	stock_index = pd.DataFrame({'Company Name': [i.name for i in company_stock_profiles],
								'Symbol': [i.symbol for i in company_stock_profiles],
								'Open': [i.open for i in company_stock_profiles], 
								'Low': [i.low for i in company_stock_profiles],
								'High': [i.high for i in company_stock_profiles],
								'Close': [i.close for i in company_stock_profiles]
								})
	return stock_index.append(Total)



se_df = stock_exchange(100)
print(se_df)


                    Company Name Symbol  ...         High       Close
0   Jackson, Williams and Porter   JAWI  ...      3568.63     2438.89
1                       Long Ltd   LOLT  ...      117.782      82.248
2      Freeman, Craig and Fisher   FRCR  ...      1479.94     975.399
3         Williams, Ruiz and Lee   WIRU  ...      976.037     631.922
4                 Holmes-Andrade   HODE  ...       35.655       19.95
..                           ...    ...  ...          ...         ...
96                  Barnes-Jones   BAES  ...         9.11       4.951
97        Fuller, Cook and Lucas   FUCO  ...       12.615       7.899
98                    Patton LLC   PALL  ...       40.693      28.096
99   Salazar, Moore and Richards   SAMO  ...       33.272      20.666
0                    GRAND TOTAL         ...  = 10463.012  = 6872.626

[101 rows x 6 columns]


In [ ]:
se_df.describe()

,Company Name,Symbol,Open,Low,High,Close
count,101,101,101,101.000,101.000,101.00
unique,101,101,100,101.000,101.000,101.00
top,Armstrong Group,FIPL,233,6.642,53.724,84.76
freq,1,1,2,1.000,1.000,1.00


In [ ]:
se_df.head()

,Company Name,Symbol,Open,Low,High,Close
0,"Jackson, Williams and Porter",JAWI,3898,1365.89,3568.63,2438.89
1,Long Ltd,LOLT,233,44.625,117.782,82.248
2,"Freeman, Craig and Fisher",FRCR,3607,568.181,1479.94,975.399
3,"Williams, Ruiz and Lee",WIRU,3257,391.267,976.037,631.922
4,Holmes-Andrade,HODE,248,14.182,35.655,19.95
